### 2.2.1 토큰화

In [1]:
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# token to token id
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


### 2.2.2 토큰 임베딩으로 변환하기

딥러닝 모델이 텍스트 데이터를 처리하기 위해 입력으로 들어오는 토큰과 토큰 사이의 관계를 계산해야함.<br>
즉, 토큰의 의미를 숫자로 표현해야함 <br>
ID는 하나의 숫자일 뿐 토큰의 의미를 담을 수 없음 <br>
최소 2개 이상의 숫자 집한인 vector로 표현되어야함 <br>
=> 이것이 임베딩

In [2]:
import torch
import torch.nn as nn

embedding_dim = 16 # 토큰하나를 16차원으로 변환
# embedding layer 만들기
embed_layer = nn.Embedding(len(str2idx), embedding_dim) # len(str2idx) 사전의 크기
input_embeddings = embed_layer(torch.tensor(input_ids)) # 토큰을 임베딩 층을 통해 임베딩으로 변환
input_embeddings = input_embeddings.unsqueeze(0) # 차원 확장
input_embeddings.shape

torch.Size([1, 5, 16])

- embedding layer는 토큰의 의미를 담아 벡터로 변환하는 단계가 아님
- embedding layer가 단어의 의미를 담기 위해서 딥러닝 모델이 학습 데이터로 훈련되어야 함
- 여기서 ML과 DL의 차이점
    - DL은 모델이 학습하는 과정에서 데이터의 의미를 잘 담은 embedding을 만드는 방법도 함께 학습함

### 2.2.3 위치 인코딩
#### RNN과 Transformer 의 차이
RNN은 입력을 순차적으로 처리하기 떄문에 순서 정보가 고려 <br>
Transformer는 모든 입력을 동시에 처리하는 과정에서 순서 정보가 사라짐 <br>
Text 에서는 순차 정보가 중요한데 Transformer를 사용하게 되면 이 역할을 인코딩이 담당 <br>

- Attention is All you need 에서 sine, cosine을 활용한 수식으로 위치에 대하 정보 입력
- 절대적 위치 인코딩 (Absolute position encoding)
    - 위치 인코딩도 위치에 따른 임베딩 층을 추가해 학습 데이터를 통해 학습 -> 모델로 추론을 수행
    - 수식을 통해 위치 정보를 추가하는 방식
    - 임베딩으로 위치 정보를 학습 하는 방식
    - 간단하게 구현 가능
    - 토큰과 토큰 사이의 상대적인 위치 정보는 활용하지 않음
    - 긴 텍스트를 추론하는 경우 성능이 떨어짐
- 상대적 위치 인코딩 (Relative position encoding)

In [3]:
# 위치 정보를 학습하는 방식의 절대적 위치 인코딩
embedding_dim = 16
max_position = 12 # 최대 토큰 수
embed_layer = nn.Embedding(len(str2idx), embedding_dim) # 토큰 임베딩 층
position_embed_layer = nn.Embedding(max_position, embedding_dim) # 위치 임베딩 층

# 위치 인덱스에 따라 임베딩을 더하도록 구현
position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
# 1. 위치 임베딩
# 위치 인덱스를 임베딩 층에 넣기만 했는데 왜 숫자가 늘어나지? max_position은 어떻게 적용되는거지?
position_encodings = position_embed_layer(position_ids)
# 2. 토큰 임베딩
# torch.tensor(input_ids) 는 차원이 확장되지 않은 것. 왜 토큰 임베딩은 왜 이렇게 넣지?
token_embeddings = embed_layer(torch.tensor(input_ids))  # (5, 16)
token_embeddings = token_embeddings.unsqueeze(0) # (1, 5, 16)
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

## 2.3 어텐션 이해하기
텍스트를 처리하는 관점에서 어떤 단어가 서로 관련이 깊은지 파악. <br>
핵심 개념: 쿼리, 키, 값 <br>
<br>
### 2.3.1 사람이 글을 읽는 방법과 어텐션
단어와 단어 사이의 관계를 계산해서 그 값에 관련이 깊은 단어와 그렇지 않은 단어를 구분 <br>
관련이 깊은 단어는 더 많이, 관련이 적은 단어는 더 적게 맥락을 반영 <br>
<br>
### 2.3.2 쿼리, 키, 값 이해하기
- 쿼리(query): 검색창에 입력하는 검색어
- 키(key): 문서의 제목, 본문, 저자 이름 등
- 값(value): 검색엔진이 쿼리와 관련이 깊은 키를 가진 문서를 찾아 관련도 순으로 정렬해 문서를 제공하는 것
<br>
가중치를 도입해 토큰과 토큰 사이의 관계를 계산 <br>
[그림] <br>
트랜스포머에서는 값도 토큰 임베딩을 가중치를 통해 변환함 <br>
세 가지 가중치를 통해 내부적으로 토큰과 토큰 사이의 관계를 계산해서 주변 맥락을 반영하는 방법을 학습함 <br>
쿼리와 키 사이의 관계를 계산한 관련도 값과 토큰 임베딩을 ㅇ값 가중치로 변환한 값을 가중합 <br>
[그림] <br>
### 2.3.3 코드로 보는 어텐션

In [4]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim) # 이거는 임의의 가중치 행렬인건가?
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 16)
keys = weight_k(input_embeddings) # (1, 5, 16)
values = weight_v(input_embeddings) # (1, 5, 16)

어텐션 연산 구현
- 스케일 점곱 방식 사용
    - 쿼리와 키를 곱함
    - 분산이 커지는 것을 방지하기 위해 임베딩 차원 수(dim_k)의 제곱근으로 나눔
    - 쿼리와 키를 곱해 계산한 스코어(scores)를 합이 1이 되도록 소프트맥스(softmax)를 취해 가중치로 바꿈
    - 가중치와 값을 곱해 입력과 동일한 형태의 출력 반환

In [5]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_casual=False):
    dim_k = querys.size(-1) #16
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return weights @ values

In [6]:
print("원본 입력 형태: ", input_embeddings.shape)
after_attention_embeddings = compute_attention(querys, keys, values) # 셀프어텐션
print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


In [7]:
# 어텐션 연산과정에 대한 클래스
class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_casual=False):
        super().__init__()
        self.is_casual = is_casual
        self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
        self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
        self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층
    
    def forward(self, querys, keys, values):
        outputs = compute_attention(
            self.weight_q(querys),
            self.weight_k(keys),
            self.weight_v(values),
            is_casual=self.is_casual
        )
        return outputs

attention_head = AttentionHead(embedding_dim, head_dim) #  입력과 출력의 차원이 같도록 둘 다 embedding_dim 전달
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings) # 왜 똑같은 input_embeddings를 query, key, value에 넣지?

### 2.3.4 멀티 헤드 어텐션
한 번에 하나의 어텐션만 수행하지 않고 여러 어텐션 연산을 동시에 적용하면 성능을 더 높일 수 있음 <br>
토큰 사이의 관계를 한 가지 측변에서 이해하기보다 여러 측면을 동시에 고려할 때 언어나 문장에 대한 이해도가 높아짐 <br>
- 헤드의 수 만큼 연산을 수행하기 위해 쿼리, 키, 값을 n_head 개로 쪼갬
- 각각의 어텐션을 계산
- 입력과 같은 형태로 다시 변환
- 선형층을 통과시켜 최종결과 반환

In [8]:
# multi head attention
class MultiheadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, n_head, is_casual=False): # d_model: 각 head의 차원
        super().__init__()
        self.n_head = n_head
        self.is_casual = is_casual
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)
        
    def forward(self, querys, keys, values):
        B, T, C = querys.size()
        # 쿼리, 키, 값을 n_head개로 쪼개기
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # multihead 로 분리
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        # 각각의 어텐션을 계산
        attention = compute_attention(querys, keys, values, self.is_casual)
        # 입력과 같은 형태로 다시 변환
        output = attention.transpose(1, 2).contiguous().view(B, T, C)
        # 선형층을 통과 후 최종 결과 반환
        output = self.concat_linear(output)
        return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

## 2.4 정규화와 피드 포워드 층
### 2.4.1 층 정규화 이해하기
딥러닝 모델의 각 층을 거치면서 데이터의 분포가 달라짐 <br>
층이 깊은 모델에서는 분포의 차이가 발생할 가능성이 높아지며 학습이 잘 되지 않음 <br>
딥러닝 모델에서 층과 층 사이에 정규화를 추가해 학습을 안정적으로 만드는 기법을 사용함 <br>
<br>

*배치 정규화* <br>
- 이미지 처리에서 많이 사용
- 모델에 입력으로 들어간는 미니 배치 사이에 정규화를 수행
- 자연어 처리에서 잘 사용하지 않는 이유
    - 입력으로 들어가는 문장의 길이가 다양해서 배치 정규화를 사용할 경우 정규화에 포함되는 데이터의 수가 제각각이라 정규화를 보장하기 어려움
<br>

*층 정규화* <br>
- 자연어 처리에서 많이 사용
- 토큰 임베딩의 평균과 표준편차를 구해 정규화를 수행
- 실제 데이터 수가 달라도 각각의 토큰 임베딩별로 정규화를 수행하기 때문에 정규화 효과에 차이가 없음
<br>

_**층 정규화의 종류**_
- *사후 정규화*: 피드 포워드 층 이후에 하는 정규화
- *사전 정규화*: 층 정규화를 적용하고 어텐션과 피드 포워드 층을 통과

In [9]:
# 층 정규화
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
print(norm_x.shape)

norm_x.mean(dim=-1).data, norm_x.std(dim=-1)

torch.Size([1, 5, 16])


(tensor([[ 6.7055e-08,  0.0000e+00, -2.2352e-08,  7.8231e-08,  1.4901e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]], grad_fn=<StdBackward0>))

### 2.4.2 피드 포워드 층
데이터의 특징을 학습하는 완전 연결 층(fully connected layer)을 말함 <br>
단어 사이의 관계를 파악하는 역할 = 멀티 헤드 어텐션 <br>
입력 텍스트 전체를 이해하는 역할 = 피드 포워드 층 <br>
<br>

_**피드포워드 층**_: 선형 층, 드롭아웃 층, 층 정규화, 활성 함수로 구성 <br>
- 임베딩 차원을 동이랗게 유지해야 쉽게 층을 쌓아 확장이 가능함 -> 입/풀력의 형태가 동일하도록 맞춤
- d_model 차원에서 d_model보다 2~3배 큰 dim_feedforward차원으로 확장했다가 다시 d_model로 변환

In [10]:
class PreLayerNormFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()
        # 왜 선형층 2개를 dimension을 다르게 해서 만듬?
        self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
        self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
        self.dropout1 = nn.Dropout(dropout) # 드롭아웃 층 1
        self.dropout2 = nn.Dropout(dropout) # 드롭아웃 층 2
        self.activation = nn.GELU() # 활성함수
        self.norm = nn.LayerNorm(d_model) # 층 정규화
        
    def forward(self, src):
        x = self.norm(src)
        x = x + self.linear2(self.dropout1(self.activation(self.linear1)))
        x = self.dropout2(x)
        return x

## 2.5 인코더
멀티 헤드 어텐션, 층 정규화, 피드 포워드 층이 반복되는 형태 <br>
트랜스포머의 인코더는 그림의 인코더 블록을 반복해서 쌓아 만듬<br>
_잔차연결(residual connection)_
- 안정적인 학습이 가능하도록 도와줌
- 입력을 다시 더해주는 형태로 구현함
<br>

_**인코더 층**_
- 정규화를 함
- 멀티 헤드 어텐션 연산 수행
- 잔차 연결을 위해 어텐션 결과에 드롭아웃 후 입력과 더해줌
- 피드 포워드 연산

In [11]:
# 인코드 층
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout):
        super().__init__()
        self.attn = MultiheadAttention(d_model, d_model, nhead) # multihead attention
        self.norm1 = nn.LayerNorm(d_model) # layer normalization
        self.dropout1 = nn.Dropout(dropout) # dropout
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # feed forward
        
    def forward(self, src):
        norm_x = self.norm1(src) # 정규화
        attn_output = self.attn(norm_x, norm_x, norm_x) # 멀티헤드 어텐션 - 어텐션의 인풋이 왜 이런지 아직도 이해가 안감
        x = src + self.dropout1(attn_output) # 잔차연결
        
        # 피드포워드
        x = self.feed_forward(x)
        return x

In [12]:

# 인코더 층이 N 번 반복되도록 구현
import copy
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layer):
        super().__init__()
        self.layers = get_clones(encoder_layer, num_layer)
        self.num_layers = num_layer
        self.norm = norm
        
    def forward(self, src):
        output = src
        for mod in self.layers:
            output = mod(output)
        return output


## 2.6 디코더
- 마스크 멀티 헤드 어텐션을 사용
- 생성을 담당하는 부분
- 앞에서 생성한 토큰을 기반으로 다음 토큰을 생성 -> 인과적(causal) 또는 자기 회귀적(auto-regressive)
<br>

_**마스크**_
- 실제 텍스트를 생성할 때 디코더는 이전까지 생성한 텍스트만 확인할 수 있음
- 학습할 때는 인코더와 디코더 모두 완성된 텍스트를 입력으로 받음
- 어텐션을 그대로 활용하면 미래 시점에 작성해야 하는 텍스트를 미리 확인하게 되는 문제가 생김
- 이것을 막기 위해 특정 시점에는 그 이전에 생성된 토큰까지만 확인할 수 있도록 마스크를 추가

In [13]:
# 디코더에서 어텐션 연산(마스크 어텐션)
def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1) # 16
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    if is_causal:
        query_length = querys.size(-2)
        key_length = keys.size(-2)
        # 미래 시점의 토큰을 제거하기 위한 마스크를 만드는 부분
        temp_mask = torch.ones(query_length, key_length, 
                               dtype=torch.bool).tril(diagnoal=0)
        scores = scores.masked_fill(temp_mask == False, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    return weights